In [195]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize

#%matplotlib qt
from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
import vcm
from pyreport import PlotUtil
from notebooks.optimization_problems.constraints import Requirements
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [196]:
from pymoo.util.function_loader import is_compiled

print('Compiled Extensions: ', is_compiled())

Compiled Extensions:  True


In [197]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [198]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [199]:
from problems.bandwidth_problem import BandwidthProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (20., 20.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()

T_range_dict_list = [{'name': 'high', 'T_min': 50e9, 'T_max': -1},
                {'name': 'low', 'T_min': 0, 'T_max': 5e9}]

T_range = T_range_dict_list[0]

requirements.min_throughput = T_range['T_min']
requirements.max_throughput = T_range['T_max']

problem = BandwidthProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 100, 'n_offsprings': 25}

from pymoo.algorithms.nsga2 import NSGA2

algorithm = NSGA2(
    pop_size=algo_settings['pop_size'],
    n_offsprings=algo_settings['n_offsprings'],
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

In [200]:
n_gen = 0

In [201]:
if False:
    n_gen = n_gen + 5000

    algorithm.has_terminated = False

    res = minimize(problem,
                   algorithm,
                   ('n_gen', n_gen),
                   seed=1,
                   copy_algorithm=False,
                   #save_history=True,
                   verbose=True
                   )

    print('Processes:', res.exec_time)
    print("Best solution found: %s" % res.X)

    import pickle

    pickle.dump(res, open('bandwidth_res_%s.pkl' % (T_range['name']), 'wb'))

In [202]:
res = pickle.load(open('bandwidth_res_%s.pkl' % (T_range['name']), 'rb'))

In [203]:
x_pass = res.X[:, problem.x_indices['pass']].astype('bool')
x_Ptx_dBm = res.X[:, problem.x_indices['power']].astype('float64')
x_Ptx_dBm[~x_pass] = np.NaN
x_Ptx_dBm = np.nanmax(x_Ptx_dBm, axis=1)
#x_modcod = np.squeeze(res.X[:, problem.x_indices['modcod']].astype('int64'))

f_energy = res.F[:,1] / 1e3 # Kilo Joule
f_throughput = ((res.F[:,0] * -1) / 1e9)   # Gigabit

postfix = '_' + T_range['name']

In [204]:
import colorcet as cc

from matplotlib.lines import Line2D
import numpy as np
import matplotlib.pyplot as plt
#from PlotUtil import apply_report_formatting, apply_report_formatting_single
import matplotlib.colors as mcol
import matplotlib.cm as cm

isend_req = 10.69

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["c","r"])
#cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["#4a569d","#dc2424"])
#cm1 = cc.cm.linear_kryw_5_100_c67_r
#cm1 = cc.cm.CET_L18

min_Ptx = np.nanmin(x_Ptx_dBm)
max_Ptx = np.nanmax(x_Ptx_dBm)

cnorm = mcol.Normalize(vmin=min_Ptx,vmax=max_Ptx)

cpick = cm.ScalarMappable(norm=cnorm,cmap=cm1)
cpick.set_array([])

cp = cpick.to_rgba(x_Ptx_dBm.tolist())
ax.scatter(f_energy, f_throughput, c=cp, marker='.', s=1)

ax.set_xlabel("Energy used [kJ]")
ax.set_ylabel("Throughput [Gbit]")
cb = plt.colorbar(cpick,label="Maximum power [dBm]")
cb.ax.tick_params(labelsize=8)
cb.ax.yaxis.label.set_size(8)

PlotUtil.apply_report_formatting(4.72, 2.76)
plt.tight_layout()

fig.savefig('D:/git/thesis_report_ae/figures/mc_pareto%s.pdf' % postfix)
plt.show()
plt.close()

In [205]:
x_bandwidth = res.X[:, problem.x_indices['bandwidth']].astype('int')
B_Hz = sys_param.B_Hz_array[np.squeeze(x_bandwidth)] / 1e6

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(B_Hz, f_throughput, marker='.', s=2)
#ax.set_xscale('log')
ax.set_xlabel("Bandwidth [MHz]")
ax.set_ylabel("Throughput [Gbit]")
#ax.set_xlim((, 11))
#ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [206]:
fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

ax.scatter(np.sum(x_pass, axis=1), f_throughput, marker='.', s=1)
ax.set_xlabel("Number of passes used")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim((0, 20))
#ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()

#fig.savefig('D:/git/thesis_report_ae/figures/vcm_modcod_vs_throughput.pdf')

In [207]:
from notebooks.optimization_problems.design_vector import explode_design_vector
from models import multi_carrier
from numba.typed import List

# recompute modcods for the solutions
x_modcod = -1*np.ones((res.X.shape[0], len(problem.x_indices['power'])))

for i in range(res.X.shape[0]):

    x = np.squeeze(res.X[i,:])

    # Explode design vector
    design_vector = explode_design_vector(x, problem.N_passes, problem.x_indices)

    x_pass = design_vector['pass'].astype('bool')
    Ptx_dBm_array = design_vector['power'][x_pass].astype('float64')
    Gtx_dBi = design_vector['antenna'][0]
    B_Hz = problem.sys_param.B_Hz_array[int(design_vector['bandwidth'][0])]
    alpha = problem.sys_param.alpha_array[0]
    carriers = multi_carrier.get_sub_carriers(B_Hz)
    EsN0_req_dB_array = np.squeeze(problem.sys_param.EsN0_req_dB_array[:, carriers - 1])
    eta_bitsym_array = np.squeeze(problem.sys_param.eta_bitsym_array[:, carriers - 1])
    eta_maee_array = np.squeeze(problem.sys_param.eta_maee_array[:, carriers - 1])

    pass_inds = np.nonzero(x_pass)[0]

    _, _, _, _, x_modcod[i,pass_inds] = vcm.compute_passes_throughput(
            pass_inds, List(problem.tof_s_list), List(problem.fspl_dB_list),
            Ptx_dBm_array, Gtx_dBi, problem.sys_param.GT_dBK, B_Hz, alpha,
            EsN0_req_dB_array, eta_bitsym_array, problem.sys_param.margin_dB)

#%matplotlib
plt.ion()
fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)

# for i in range(res.X.shape[0]):
#     x = x_modcod[i, :]
#     x = x[x > -1]
#     #ax.scatter(x, f_throughput[i].repeat(len(x)), color='b', marker='.', s=2)
#     ax.plot((np.min(x), np.max(x)), (f_throughput[i], f_throughput[i]), color='tab:blue')

x_modcod[x_modcod == -1] = np.NaN
avg_modcod = np.floor(np.nanmean(x_modcod, axis=1))
ind_sort = np.argsort(f_throughput)
ax.plot(avg_modcod[ind_sort], f_throughput[ind_sort], color='tab:blue')

#ax.set_xscale('log')
ax.set_xlabel("Modcod")
ax.set_ylabel("Throughput [Gbit]")
ax.set_xlim((0, 28))
#ax.set_ylim((0, 1200))

PlotUtil.apply_report_formatting(3.146, 2.76)
plt.tight_layout()